<a href="https://colab.research.google.com/github/priyanshusharmaghec20ee-ui/Alternate-product-recommendation-system-using-ai/blob/main/main_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df2 = pd.DataFrame(pd.read_csv('/content/df_amazon_combined.csv'))

In [3]:
df2['ratings'] = pd.to_numeric(df2['ratings'], errors='coerce').fillna('no rating')
df2['no_of_ratings'] = df2['no_of_ratings'].astype(str).str.extract(r'(\d+)', expand=False)
df2['no_of_ratings'] = pd.to_numeric(df2['no_of_ratings'], errors='coerce').fillna(0).astype(int)

df2['discount_price'] = df2['discount_price'].astype(str).str.replace('₹', '', regex=False).str.replace(',', '', regex=True).str.extract(r'(\d+\.?\d*)', expand=False).astype(float)
df2['actual_price'] = df2['actual_price'].astype(str).str.replace('₹', '', regex=False).str.replace(',', '', regex=True).str.extract(r'(\d+\.?\d*)', expand=False).astype(float)

df2['discount_price'] = df2['discount_price'].fillna(df2['discount_price'].median())
df2['actual_price'] = df2['actual_price'].fillna(df2['actual_price'].median())


In [4]:
df = pd.DataFrame(pd.read_csv('/content/flipkart_com-ecommerce_sample.csv'))

In [5]:
df = df.drop_duplicates().reset_index(drop = True)

In [6]:
df['retail_price'] = df['retail_price'].fillna('')


In [7]:
df['retail_price'] = pd.to_numeric(df['retail_price'], errors='coerce')
df['discounted_price'] = pd.to_numeric(df['discounted_price'], errors='coerce')

df['retail_price'] = df['retail_price'].fillna(df['retail_price'].median())
df['discounted_price'] = df['discounted_price'].fillna(df['discounted_price'].median())

df['brand'] = df['brand'].fillna('unknown')

df['image'] = df['image'].fillna('')
df['description'] = df['description'].fillna('')
df['product_specifications'] = df['product_specifications'].fillna('')

In [8]:
df.isnull().sum()

,0
uniq_id,0
crawl_timestamp,0
product_url,0
product_name,0
product_category_tree,0
pid,0
retail_price,0
discounted_price,0
image,0
is_FK_Advantage_product,0


In [9]:
df = df[['product_name','product_category_tree',  'discounted_price', 'description', 'product_rating', 'brand', 'product_specifications', 'retail_price', 'image','product_url']]

In [10]:
import ast

df = df.rename(columns={
    'pid': 'product_id',
    'product_name': 'title',
    'product_category_tree': 'category',
    'description': 'description',
    'discounted_price': 'price',
    'product_rating': 'rating',
    'brand': 'brand',
    'image': 'image_url'
})

def extract_first_image_url(image_url_str):
    if pd.isna(image_url_str) or image_url_str == '':
        return ''
    try:
        url_list = ast.literal_eval(image_url_str)
        if isinstance(url_list, list) and len(url_list) > 0:
            return url_list[0]
        elif isinstance(image_url_str, str):
            return image_url_str
    except (ValueError, SyntaxError):
        return image_url_str

df['image_url'] = df['image_url'].apply(extract_first_image_url)

if 'product_url' not in df.columns:
    df['product_url'] = None

df['platform'] = "Flipkart"

In [11]:
df['product_id'] = df.index.astype(str)
df['rating'] = df['rating'].replace('No rating available', None)
df['rating'] = pd.to_numeric(df['rating'], errors='coerce').fillna(0.0)

In [12]:
import re


cleaned_text_pattern = re.compile(r'[^a-z\s]')

def preprocess_text(text):
    text = text.lower()
    text = cleaned_text_pattern.sub('', text)
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

print("Preprocess_text function updated to use pre-compiled regex.")

Preprocess_text function updated to use pre-compiled regex.


In [13]:
query_embedding_cache = {}

def get_query_embedding(query, model):
    if query in query_embedding_cache:
        return query_embedding_cache[query]

    tokenized_query = preprocess_text(query)
    embeddings = [model.wv[word] for word in tokenized_query if word in model.wv]

    if embeddings:
        query_embedding = np.mean(embeddings, axis=0)
    else:
        query_embedding = np.zeros(model.vector_size)

    query_embedding_cache[query] = query_embedding
    return query_embedding

print("get_query_embedding function updated with caching.")

get_query_embedding function updated with caching.


In [14]:
df['no_of_ratings'] = 0
target_columns = ['product_id', 'title', 'category', 'description', 'price', 'retail_price', 'rating', 'no_of_ratings', 'brand', 'image_url', 'product_url', 'platform']
df_flipkart = df[target_columns]

In [15]:
df2['product_id'] = df2.index.astype(str)
print("Added 'product_id' column to df2.")

Added 'product_id' column to df2.


In [16]:
df2 = df2.rename(columns={
    'name': 'title',
    'discount_price': 'price',
    'actual_price': 'retail_price',
    'ratings': 'rating',
    'image': 'image_url',
    'link': 'product_url'
})
print("Renamed columns in df2.")

Renamed columns in df2.


In [17]:
df2['category'] = df2['main_category'].astype(str) + ' >> ' + df2['sub_category'].astype(str)
print("Created 'category' column by concatenating 'main_category' and 'sub_category'.")

Created 'category' column by concatenating 'main_category' and 'sub_category'.


In [18]:
df2['rating'] = pd.to_numeric(df2['rating'], errors='coerce').fillna(0.0)
print("Ensured 'rating' column is numeric and filled NaN values with 0.0.")

Ensured 'rating' column is numeric and filled NaN values with 0.0.


In [19]:
df2['description'] = ''
df2['brand'] = 'unknown'
df2['product_specifications'] = ''
df2['platform'] = 'Amazon'
print("Added 'description', 'brand', 'product_specifications', and 'platform' columns with default values.")

Added 'description', 'brand', 'product_specifications', and 'platform' columns with default values.


In [20]:
target_columns = ['product_id', 'title', 'category', 'description', 'price', 'retail_price', 'rating', 'no_of_ratings', 'brand', 'image_url', 'product_url', 'platform']
df_amazon = df2[target_columns]
print("Selected and reordered columns for df_amazon.")

Selected and reordered columns for df_amazon.


In [21]:
df_master = pd.concat([df_flipkart, df_amazon], axis=0)
df_master.reset_index(drop=True, inplace=True)



In [22]:
df_master.head(3)

,product_id,title,category,description,price,retail_price,rating,no_of_ratings,brand,image_url,product_url,platform
0,0,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,379.0,999.0,0.0,0,Alisha,http://img5a.flixcart.com/image/short/u/4/a/al...,http://www.flipkart.com/alisha-solid-women-s-c...,Flipkart
1,1,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,22646.0,32157.0,0.0,0,FabHomeDecor,http://img6a.flixcart.com/image/sofa-bed/j/f/y...,http://www.flipkart.com/fabhomedecor-fabric-do...,Flipkart
2,2,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",Key Features of AW Bellies Sandals Wedges Heel...,499.0,999.0,0.0,0,AW,http://img5a.flixcart.com/image/shoe/7/z/z/red...,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,Flipkart


In [23]:
df_master = df_master.drop(df_master.sample(n=90000, random_state=42).index)

In [24]:
df_master.shape

(28597, 12)

In [25]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

In [26]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
s_w = set(stopwords.words('english'))
lem = WordNetLemmatizer()

In [28]:
def preprocess_text(text):
  text = str(text).lower()
  text = re.sub(r'[^a-zA-Z\s]', '', text)

  words = text.split()
  words = [i for i in words if i not in s_w]
  words = [lem.lemmatize(i) for i in words]

  return " ".join(words)


In [29]:
df_master['tag'] = df_master['title'].astype(str) + ' ' + df_master['category'].astype(str) + ' ' + df_master['description'].astype(str) + ' ' + df_master['brand'].astype(str)
df_master['tag'] = df_master['tag'].apply(preprocess_text)

In [30]:
df_master = df_master.reset_index(drop = True)

In [31]:
indices = pd.Series(df_master.index,index = df_master['title']).drop_duplicates()
indices

,0
title,
AW Bellies,0
Eternal Gandhi Super Series Crystal Paper Weights with Silver Finish,1
Freelance Vacuum Bottles 350 ml Bottle,2
Alisha Solid Women's Cycling Shorts,3
Carrel Printed Women's,4
...,...
"Lifeline Exercise Motorized Treadmill DK 1000 for Weight Loss at Home | Bundles with Silicon Spray, Fitness Ball (75cm) an...",28592
Colado Fitness Exercise Cycle CFE-101 with Back Rest,28593
Colado Fitness Moving Handle AIR Bike Black,28594


In [32]:
df_master['tokenized_tag'] = df_master['tag'].apply(lambda x: x.split())

In [33]:
import sys
!{sys.executable} -m pip uninstall -y gensim smart_open
!{sys.executable} -m pip install gensim
from gensim.models import Word2Vec

Found existing installation: gensim 4.4.0
Uninstalling gensim-4.4.0:
  Successfully uninstalled gensim-4.4.0
Found existing installation: smart_open 7.5.1
Uninstalling smart_open-7.5.1:
  Successfully uninstalled smart_open-7.5.1
  Using cached gensim-4.4.0-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (8.4 kB)
  Using cached smart_open-7.5.1-py3-none-any.whl.metadata (24 kB)
Using cached gensim-4.4.0-cp312-cp312-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (27.9 MB)
Using cached smart_open-7.5.1-py3-none-any.whl (64 kB)


In [34]:
word2vec_model = Word2Vec(vector_size=25, window=5, min_count=1, sg=1, workers=4)
word2vec_model.build_vocab(df_master['tokenized_tag'])
word2vec_model.train(df_master['tokenized_tag'], total_examples=word2vec_model.corpus_count, epochs=10)

print("Word2Vec model trained successfully.")

Word2Vec model trained successfully.


In [35]:
def get_product_embedding(words):
    vectors = [
        word2vec_model.wv[word]
        for word in words
        if word in word2vec_model.wv.key_to_index
    ]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

df_master['product_embedding'] = df_master['tokenized_tag'].apply(get_product_embedding)

In [36]:
!pip install faiss-cpu

In [37]:
import faiss

product_embeddings = np.array(df_master['product_embedding'].tolist()).astype('float32')

embedding_dim = product_embeddings.shape[1]

print(f"Product embeddings shape: {product_embeddings.shape}")
print(f"Embedding dimension: {embedding_dim}")

Product embeddings shape: (28597, 25)
Embedding dimension: 25


In [38]:
import faiss

index = faiss.IndexFlatL2(embedding_dim)

index.add(product_embeddings)

print(f"FAISS index created with {index.ntotal} vectors.")

FAISS index created with 28597 vectors.


In [39]:
def recommend_faiss(query, n=10):

    tokenized_query = preprocess_text(query).split()

    query_embedding = get_product_embedding(tokenized_query)

    query_embedding_faiss = query_embedding.reshape(1, -1).astype('float32')

    distances, indices = index.search(query_embedding_faiss, n)

    similar_indices = indices.flatten()

    recommended_products = df_master.iloc[similar_indices].copy()


    similarity_scores = 1 / (1 + distances.flatten())
    recommended_products['similarity score'] = similarity_scores


    return recommended_products[['title', 'brand', 'price', 'rating', 'platform', 'image_url', 'product_url', 'similarity score']]

print("recommend_faiss function defined successfully.")

recommend_faiss function defined successfully.


In [40]:
from IPython.display import display, HTML
import pandas as pd


faiss_recommended_df = recommend_faiss('headphones', n=5)

def make_clickable(val):
    if pd.isna(val) or val == '':
        return ''
    return f'<a href="{val}" target="_blank">Link</a>'

styled_faiss_df = faiss_recommended_df.style.format({'product_url': make_clickable})

display(styled_faiss_df)
print("FAISS-based recommendations displayed successfully.")

,title,brand,price,rating,platform,image_url,product_url,similarity score
4454,signature VM-49 UNIVERSAL HANDSFREE Headphones,signature,249.000000,0.000000,Flipkart,http://img5a.flixcart.com/image/headphone/universal-handsfree/f/z/y/signature-vm-49-1100x1100-imaejrxy9rffhffj.jpeg,Link,0.108422
1963,LIFE LIKE VM-42 3D SOUND WIRED HEADPHONES Headphones,LIFE LIKE,449.000000,0.000000,Flipkart,http://img5a.flixcart.com/image/headphone/3d-sound-wired-headphones/g/j/2/life-like-vm-42-original-imaegbsqradwebr8.jpeg,Link,0.105646
1991,LIFE LIKE OTP-200 WITH MIC Wireless Bluetooth Headset,LIFE LIKE,599.000000,0.000000,Flipkart,http://img6a.flixcart.com/image/headset/w/3/t/life-like-otp-200-with-mic-original-imaeggayagknzqsj.jpeg,Link,0.104655
325,DEBOCK DEBOCK Earphone For HTC Desire 320 Stereo Dynamic Headphone Wired Headphones,DEBOCK,320.000000,0.000000,Flipkart,http://img6a.flixcart.com/image/headphone/stereo-dynamic-headphone/t/c/9/debock-debock-earphone-for-motorola-moto-g-dual-sim-3rd-gen-1100x1100-imaehfs7rea8zhju.jpeg,Link,0.104369
4624,THERISE MD0005 Wired Headset,THERISE,249.000000,0.000000,Flipkart,http://img6a.flixcart.com/image/headset/q/h/h/therise-md0009-1100x1100-imae82uu2svufc9y.jpeg,Link,0.103315


FAISS-based recommendations displayed successfully.


In [41]:
import faiss

faiss.write_index(index, "faiss_index.bin")
print("FAISS index saved as faiss_index.bin")

FAISS index saved as faiss_index.bin


In [42]:
df_master.to_pickle("df_master.pkl")
print("df_master saved as df_master.pkl")

df_master saved as df_master.pkl


In [43]:
import pickle

with open("word2vec_model.pkl", "wb") as f:
    pickle.dump(word2vec_model, f)
print("Word2Vec model saved as word2vec_model.pkl")

Word2Vec model saved as word2vec_model.pkl


In [44]:
indices.to_pickle("indices.pkl")
print("indices saved as indices.pkl")

indices saved as indices.pkl
